# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from my_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [8]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2356151356980,
 'week52high': 181.88,
 'week52low': 128.65,
 'week52highSplitAdjustOnly': 182.94,
 'week52lowSplitAdjustOnly': 129.04,
 'week52change': -0.04999961515074536,
 'sharesOutstanding': 15908118000,
 'float': 0,
 'avg10Volume': 59874072,
 'avg30Volume': 86158128,
 'day200MovingAvg': 149.46,
 'day50MovingAvg': 145.57,
 'employees': 147000,
 'ttmEPS': 6.11,
 'ttmDividendRate': 0.9079288885940536,
 'dividendYield': 0.006130098498373193,
 'nextDividendDate': '',
 'exDividendDate': '2022-11-04',
 'nextEarningsDate': '2023-01-26',
 'peRatio': 23.60802137190265,
 'beta': 1.2718484691361442,
 'maxChangePercent': 56.34474214031284,
 'year5ChangePercent': 2.5607901968279614,
 'year2ChangePercent': 0.29180305563623343,
 'year1ChangePercent': -0.080094108527902,
 'ytdChangePercent': -0.16106565223449598,
 'month6ChangePercent': 0.0572285351056836,
 'month3ChangePercent': -0.12747336646472063,
 'month1ChangePercent': -0.0261539887841028,
 'day30C

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [9]:
data['year1ChangePercent']

-0.080094108527902

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            final_dataframe = final_dataframe.append(
                                            pd.Series([symbol, 
                                                       data[symbol]['quote']['latestPrice'],
                                                       data[symbol]['stats']['year1ChangePercent'],
                                                       'N/A'
                                                       ], 
                                                      index = my_columns), 
                                            ignore_index = True)
            
final_dataframe

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/290473506.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.96,0.032655,N/A
1,AAL,14.50,-0.254882,N/A
2,AAP,150.00,-0.338298,N/A
3,AAPL,148.11,-0.080094,N/A
4,ABBV,159.62,0.397597,N/A
...,...,...,...,...
496,YUM,126.53,0.01641,N/A
497,ZBH,119.21,-0.079016,N/A
498,ZBRA,271.90,-0.545333,N/A
499,ZION,51.75,-0.212105,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [12]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,N/A
1,OXY,70.28,1.210118,N/A
2,MPC,125.20,0.990075,N/A
3,VLO,138.40,0.965203,N/A
4,FTI,12.10,0.905512,N/A
5,XOM,113.21,0.854145,N/A
6,MRO,30.38,0.827577,N/A
7,HES,144.76,0.796444,N/A
8,COP,126.81,0.772587,N/A
9,HRB,43.10,0.753822,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [15]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000
10000


In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,2
1,OXY,70.28,1.210118,2
2,MPC,125.20,0.990075,1
3,VLO,138.40,0.965203,1
4,FTI,12.10,0.905512,16
5,XOM,113.21,0.854145,1
6,MRO,30.38,0.827577,6
7,HES,144.76,0.796444,1
8,COP,126.81,0.772587,1
9,HRB,43.10,0.753822,4


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [25]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_string)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            hqm_dataframe = hqm_dataframe.append(
                                            pd.Series([symbol, 
                                                       data[symbol]['quote']['latestPrice'],
                                                       'N/A',
                                                       data[symbol]['stats']['year1ChangePercent'],
                                                       'N/A',
                                                       data[symbol]['stats']['month6ChangePercent'],
                                                       'N/A',
                                                       data[symbol]['stats']['month3ChangePercent'],
                                                       'N/A',
                                                       data[symbol]['stats']['month1ChangePercent'],
                                                       'N/A',
                                                       'N/A'
                                                       ], 
                                                      index = hqm_columns), 
                                            ignore_index = True)
        
hqm_dataframe

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/dk/gjdplyjn48bfrls46rqgx28m0000gn/T/ipykernel_54037/3173093505.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.96,N/A,0.032655,N/A,0.308442,N/A,0.15603,N/A,0.166989,N/A,N/A
1,AAL,14.50,N/A,-0.254882,N/A,-0.101054,N/A,0.005548,N/A,0.014696,N/A,N/A
2,AAP,150.00,N/A,-0.338298,N/A,-0.176169,N/A,-0.163849,N/A,-0.173098,N/A,N/A
3,AAPL,148.11,N/A,-0.080094,N/A,0.057229,N/A,-0.127473,N/A,-0.026154,N/A,N/A
4,ABBV,159.62,N/A,0.397597,N/A,0.071021,N/A,0.157283,N/A,0.065412,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,126.53,N/A,0.01641,N/A,0.11053,N/A,0.093419,N/A,0.142664,N/A,N/A
497,ZBH,119.21,N/A,-0.079016,N/A,0.005009,N/A,0.067327,N/A,0.076583,N/A,N/A
498,ZBRA,271.90,N/A,-0.545333,N/A,-0.157474,N/A,-0.17641,N/A,-0.00976,N/A,N/A
499,ZION,51.75,N/A,-0.212105,N/A,-0.044087,N/A,-0.089655,N/A,0.012849,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [28]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100 if math.isnan(hqm_dataframe.loc[row, f'{time_period} Price Return']) else 0

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

TypeError: must be real number, not NoneType

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()